In [ ]:
import numpy as np
from wfdb import readsignal

In [ ]:
# Testing rdsamp format 212
sig, fields=readsignal.rdsamp('sampledata/100', sampfrom=100, sampto=9999) # Default physical values
print(sig)
print(len(sig))

sig, fields=readsignal.rdsamp('sampledata/100', sampto=9999, physical=0) # Digital values
print(sig)

In [ ]:
# Testing rdsamp format 16
sig, fields=readsignal.rdsamp('sampledata/test01_00s')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 80
sig, fields=readsignal.rdsamp('sampledata/3000003_0003')
print(sig)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
import numpy as np
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/a103l') 
print(sig)
print(fields)
#gains=np.array([1,2,3])
#sig=np.subtract(sig, gains)
#print(sig)

In [3]:
# Testing rdsamp multi-segment variable layout record (from https://physionet.org/physiobank/database/mimic2wdb/matched/s00001/)
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=20)
print("len(sig): ", len(sig))
#print(len(fields))
#print(fields[0])
print(sig)

# Return single numpy array
#sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', stacksegments=1)
#print(sig)
#print(sig.shape)

cumsumlengths:  [3261, 10011, 10136, 11490, 12002, 918261, 920865, 2225806, 2227086, 2227598, 4438365, 4610865, 10370865, 10528365, 14428365, 14432790, 14450865, 14458365, 14495865, 14518365]
readsegs: [0]
readsamps: [[10, 20]]




fields[filename]:  ['3975656_layout', '~', '3975656_0001', '3975656_0002', '3975656_0003', '3975656_0004', '3975656_0005', '~', '3975656_0006', '3975656_0007', '3975656_0008', '3975656_0009', '~', '3975656_0010', '~', '3975656_0011', '3975656_0012', '3975656_0013', '3975656_0014', '3975656_0015', '3975656_0016']
startseg:  1 



1
len(sig):  20
[3261, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [ ]:
# Testing rdsamp multi-dat signals (from https://physionet.org/physiobank/database/ptbdb/patient001/)
import numpy as np
from wfdb import readsignal

sig, fields=readsignal.rdsamp('sampledata/s0010_re', sampto=100)
print(sig)
print(fields["signame"])



In [ ]:
# Testing rdsamp multi-segment record with sampfrom and sampto
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200)


# Return single array
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, stacksegments=1)